##### Perceptron algorithm is the simplest type of artificial neural network. It is a model of a single neuron that can be used for two-class classification problems and provides the foundation for later developing much larger networks.

##### Perceptron receives input signals from examples of training data that we weight and combined in a linear equation called the activation.
##### activation = sum(weight_i * x_i) + bias

##### Activation is then transformed into an output value or prediction using a transfer function, such as the step transfer function. 
##### Ex : prediction = 1.0 if activation >= 0.0 else 0.0

##### In perceptron algorithm, each iteration the weights (w) are updated using the equation "w = w + learning_rate * (expected - predicted) * x"

In [1]:
def predict(row, weights):
    activation = weights[0]
    for i in range(len(row)-1):
        activation += weights[i]*row[i]
    return 1.0 if activation >= 0.0 else 0.0

In [2]:
# test predictions
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
weights = [-0.1, 0.20653640140000007, -0.23418117710000003]
for row in dataset:
	prediction = predict(row, weights)
	print("Expected=%d, Predicted=%d" % (row[-1], prediction))

Expected=0, Predicted=1
Expected=0, Predicted=1
Expected=0, Predicted=1
Expected=0, Predicted=1
Expected=0, Predicted=1
Expected=1, Predicted=0
Expected=1, Predicted=0
Expected=1, Predicted=0
Expected=1, Predicted=0
Expected=1, Predicted=0


In [3]:
### Dring training weights are updated using below formula:
### w(t+1)= w(t) + learning_rate * (expected(t) - predicted(t)) * x(t)
### bias(t+1) = bias(t) + learning_rate * (expected(t) - predicted(t))

In [4]:
def train_weights(train, l_rate, n_epochs):
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epochs):
        sum_error = 0.0
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            sum_error += error**2
            weights[0] = weights[0] + l_rate*error
            for i in range(len(row)-1):
                weights[i+1] = weights[i+1] + l_rate*error*row[i]
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    return weights

In [5]:
# Calculate weights
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
l_rate = 0.1
n_epoch = 5
weights = train_weights(dataset, l_rate, n_epoch)
print(weights)

>epoch=0, lrate=0.100, error=3.000
>epoch=1, lrate=0.100, error=8.000
>epoch=2, lrate=0.100, error=8.000
>epoch=3, lrate=0.100, error=8.000
>epoch=4, lrate=0.100, error=7.000
[-0.7999999999999999, 4.8825257227, -4.199594114499999]


In [6]:
# Perceptron Algorithm on the Sonar Dataset
from random import seed
from random import randrange
from csv import reader

In [8]:
def load_csv(file_name):
    dataset = list();
    with open(file_name, 'r') as f:
        csv_reader = reader(f)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
        return dataset

In [9]:
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [10]:
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique_class = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique_class):
        lookup[value] = 1
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [11]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

In [12]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [13]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [14]:
# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, l_rate, n_epoch):
	predictions = list()
	weights = train_weights(train, l_rate, n_epoch)
	for row in test:
		prediction = predict(row, weights)
		predictions.append(prediction)
	return(predictions)

In [16]:
# Test the Perceptron algorithm on the sonar dataset
seed(1)
# load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert string class to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 3
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

>epoch=0, lrate=0.010, error=0.000
>epoch=1, lrate=0.010, error=0.000
>epoch=2, lrate=0.010, error=0.000
>epoch=3, lrate=0.010, error=0.000
>epoch=4, lrate=0.010, error=0.000
>epoch=5, lrate=0.010, error=0.000
>epoch=6, lrate=0.010, error=0.000
>epoch=7, lrate=0.010, error=0.000
>epoch=8, lrate=0.010, error=0.000
>epoch=9, lrate=0.010, error=0.000
>epoch=10, lrate=0.010, error=0.000
>epoch=11, lrate=0.010, error=0.000
>epoch=12, lrate=0.010, error=0.000
>epoch=13, lrate=0.010, error=0.000
>epoch=14, lrate=0.010, error=0.000
>epoch=15, lrate=0.010, error=0.000
>epoch=16, lrate=0.010, error=0.000
>epoch=17, lrate=0.010, error=0.000
>epoch=18, lrate=0.010, error=0.000
>epoch=19, lrate=0.010, error=0.000
>epoch=20, lrate=0.010, error=0.000
>epoch=21, lrate=0.010, error=0.000
>epoch=22, lrate=0.010, error=0.000
>epoch=23, lrate=0.010, error=0.000
>epoch=24, lrate=0.010, error=0.000
>epoch=25, lrate=0.010, error=0.000
>epoch=26, lrate=0.010, error=0.000
>epoch=27, lrate=0.010, error=0.000
>e

>epoch=259, lrate=0.010, error=0.000
>epoch=260, lrate=0.010, error=0.000
>epoch=261, lrate=0.010, error=0.000
>epoch=262, lrate=0.010, error=0.000
>epoch=263, lrate=0.010, error=0.000
>epoch=264, lrate=0.010, error=0.000
>epoch=265, lrate=0.010, error=0.000
>epoch=266, lrate=0.010, error=0.000
>epoch=267, lrate=0.010, error=0.000
>epoch=268, lrate=0.010, error=0.000
>epoch=269, lrate=0.010, error=0.000
>epoch=270, lrate=0.010, error=0.000
>epoch=271, lrate=0.010, error=0.000
>epoch=272, lrate=0.010, error=0.000
>epoch=273, lrate=0.010, error=0.000
>epoch=274, lrate=0.010, error=0.000
>epoch=275, lrate=0.010, error=0.000
>epoch=276, lrate=0.010, error=0.000
>epoch=277, lrate=0.010, error=0.000
>epoch=278, lrate=0.010, error=0.000
>epoch=279, lrate=0.010, error=0.000
>epoch=280, lrate=0.010, error=0.000
>epoch=281, lrate=0.010, error=0.000
>epoch=282, lrate=0.010, error=0.000
>epoch=283, lrate=0.010, error=0.000
>epoch=284, lrate=0.010, error=0.000
>epoch=285, lrate=0.010, error=0.000
>

>epoch=7, lrate=0.010, error=0.000
>epoch=8, lrate=0.010, error=0.000
>epoch=9, lrate=0.010, error=0.000
>epoch=10, lrate=0.010, error=0.000
>epoch=11, lrate=0.010, error=0.000
>epoch=12, lrate=0.010, error=0.000
>epoch=13, lrate=0.010, error=0.000
>epoch=14, lrate=0.010, error=0.000
>epoch=15, lrate=0.010, error=0.000
>epoch=16, lrate=0.010, error=0.000
>epoch=17, lrate=0.010, error=0.000
>epoch=18, lrate=0.010, error=0.000
>epoch=19, lrate=0.010, error=0.000
>epoch=20, lrate=0.010, error=0.000
>epoch=21, lrate=0.010, error=0.000
>epoch=22, lrate=0.010, error=0.000
>epoch=23, lrate=0.010, error=0.000
>epoch=24, lrate=0.010, error=0.000
>epoch=25, lrate=0.010, error=0.000
>epoch=26, lrate=0.010, error=0.000
>epoch=27, lrate=0.010, error=0.000
>epoch=28, lrate=0.010, error=0.000
>epoch=29, lrate=0.010, error=0.000
>epoch=30, lrate=0.010, error=0.000
>epoch=31, lrate=0.010, error=0.000
>epoch=32, lrate=0.010, error=0.000
>epoch=33, lrate=0.010, error=0.000
>epoch=34, lrate=0.010, error=0

>epoch=264, lrate=0.010, error=0.000
>epoch=265, lrate=0.010, error=0.000
>epoch=266, lrate=0.010, error=0.000
>epoch=267, lrate=0.010, error=0.000
>epoch=268, lrate=0.010, error=0.000
>epoch=269, lrate=0.010, error=0.000
>epoch=270, lrate=0.010, error=0.000
>epoch=271, lrate=0.010, error=0.000
>epoch=272, lrate=0.010, error=0.000
>epoch=273, lrate=0.010, error=0.000
>epoch=274, lrate=0.010, error=0.000
>epoch=275, lrate=0.010, error=0.000
>epoch=276, lrate=0.010, error=0.000
>epoch=277, lrate=0.010, error=0.000
>epoch=278, lrate=0.010, error=0.000
>epoch=279, lrate=0.010, error=0.000
>epoch=280, lrate=0.010, error=0.000
>epoch=281, lrate=0.010, error=0.000
>epoch=282, lrate=0.010, error=0.000
>epoch=283, lrate=0.010, error=0.000
>epoch=284, lrate=0.010, error=0.000
>epoch=285, lrate=0.010, error=0.000
>epoch=286, lrate=0.010, error=0.000
>epoch=287, lrate=0.010, error=0.000
>epoch=288, lrate=0.010, error=0.000
>epoch=289, lrate=0.010, error=0.000
>epoch=290, lrate=0.010, error=0.000
>

>epoch=488, lrate=0.010, error=0.000
>epoch=489, lrate=0.010, error=0.000
>epoch=490, lrate=0.010, error=0.000
>epoch=491, lrate=0.010, error=0.000
>epoch=492, lrate=0.010, error=0.000
>epoch=493, lrate=0.010, error=0.000
>epoch=494, lrate=0.010, error=0.000
>epoch=495, lrate=0.010, error=0.000
>epoch=496, lrate=0.010, error=0.000
>epoch=497, lrate=0.010, error=0.000
>epoch=498, lrate=0.010, error=0.000
>epoch=499, lrate=0.010, error=0.000
>epoch=0, lrate=0.010, error=0.000
>epoch=1, lrate=0.010, error=0.000
>epoch=2, lrate=0.010, error=0.000
>epoch=3, lrate=0.010, error=0.000
>epoch=4, lrate=0.010, error=0.000
>epoch=5, lrate=0.010, error=0.000
>epoch=6, lrate=0.010, error=0.000
>epoch=7, lrate=0.010, error=0.000
>epoch=8, lrate=0.010, error=0.000
>epoch=9, lrate=0.010, error=0.000
>epoch=10, lrate=0.010, error=0.000
>epoch=11, lrate=0.010, error=0.000
>epoch=12, lrate=0.010, error=0.000
>epoch=13, lrate=0.010, error=0.000
>epoch=14, lrate=0.010, error=0.000
>epoch=15, lrate=0.010, er

>epoch=259, lrate=0.010, error=0.000
>epoch=260, lrate=0.010, error=0.000
>epoch=261, lrate=0.010, error=0.000
>epoch=262, lrate=0.010, error=0.000
>epoch=263, lrate=0.010, error=0.000
>epoch=264, lrate=0.010, error=0.000
>epoch=265, lrate=0.010, error=0.000
>epoch=266, lrate=0.010, error=0.000
>epoch=267, lrate=0.010, error=0.000
>epoch=268, lrate=0.010, error=0.000
>epoch=269, lrate=0.010, error=0.000
>epoch=270, lrate=0.010, error=0.000
>epoch=271, lrate=0.010, error=0.000
>epoch=272, lrate=0.010, error=0.000
>epoch=273, lrate=0.010, error=0.000
>epoch=274, lrate=0.010, error=0.000
>epoch=275, lrate=0.010, error=0.000
>epoch=276, lrate=0.010, error=0.000
>epoch=277, lrate=0.010, error=0.000
>epoch=278, lrate=0.010, error=0.000
>epoch=279, lrate=0.010, error=0.000
>epoch=280, lrate=0.010, error=0.000
>epoch=281, lrate=0.010, error=0.000
>epoch=282, lrate=0.010, error=0.000
>epoch=283, lrate=0.010, error=0.000
>epoch=284, lrate=0.010, error=0.000
>epoch=285, lrate=0.010, error=0.000
>

In [18]:
### Refference/Credits : https://machinelearningmastery.com/